## Install dependencies

In [1]:
import os
import tensorflow as tf
%matplotlib inline

from data.abc import ABCPreProcessor

## Overview of Preprocessing steps

In this notebook, we will be preprocessing two types of data --> **ABC Notation** data and **Audio** data

### ABC Notation Data

- Strip away **Tune body**, **key**, **meter**, **rhythm** and store all other fields of an ABC track as metadata
- Use key and meter as conditioning symbols when generating a tune
- Tokenize according to vocabulary of musical transcription tokens

- Create a TFRecord Dataset consisting sequence examples like --> **[ tune, meter, key, rhythm ]**

### Audio Data
- Turning the full audio into short examples (4 seconds by default, but adjustable with flags)
- Inferring the fundamental frequency (or "pitch") with CREPE
- Computing the loudness features

- Create TFRecord Dataset consisting sequence examples like --> **[ Audio, f0_feature, loudness_feature ]**

#### Each tune be indexed such that using its ID, we can find its ABC Notation as well as related audio files
- A tune can be associated with more than one audio file (Different audio lengths!!)

At the end of the file, we should merge both the datasets, to obtain a single TFRecord file containing preprocessed ABC data and preprocessed audio files indexed according to the different tunes

## Initialize common variables

In [2]:
# Mention the path to the datastore
BASE_DIR = "/home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/"
ABC_DATA_DIR = os.path.join(BASE_DIR, "datasets", "abc_data")
AUDIO_DATA_DIR = os.path.join(BASE_DIR, "datasets", "audio")
ABC_TFRECORD_DIR = os.path.join(BASE_DIR, "tfrecords", 'abc')
AUDIO_TFRECORD_DIR = os.path.join(BASE_DIR, "tfrecords", 'audio')
PROCESSED_ABC_FILENAME = 'processed-abc-files'
PROCESSED_AUDIO_FILENAME = 'processed-audio-files'

## Preprocessing - ABC Notation Dataset

#### To understand the underlying distribution of tunes in Irish music, it can be helpful to visualize the following quantities:
- Maximum length of tunes in each category
- Number of tunes in each category
- Number of tunes in each key
- Number of tunes in each meter

In [3]:
preprocessor = ABCPreProcessor(ABC_TFRECORD_DIR, PROCESSED_ABC_FILENAME)
json_path = preprocessor.process(ABC_DATA_DIR)
#preprocessor.visualize_stats()
tfrecord_path = preprocessor.save_as_tfrecord_dataset()

Processing files and writing extracted information to JSON ..
========== 0. /home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/datasets/abc_data/DoubleJigs-cleaned.abc ==========
{'tune': 'B|:AFDDFA|dedcBA|BGEEFA|B2ABcd|\\AFDDFA|dedcBA|BcdecA|[1d3dcB:|[2d3d2||g|f2d-ded|faaafd|cAAeAA|cBcefg|f2d-ded|faaafd|BcdecA|d3d2:|', 'K': 'D', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'B|:AFDDFA|dedcBA|BGEEFA|B2ABcd|\\AFDDFA|dedcBA|BcdecA|[1d3dcB:|[2d3d2||g|f2d-ded|faaafd|cAAeAA|cBcefg|f2d-ded|faaafd|BcdecA|d3d2:|', 'K': 'D', 'M': '6/8', 'R': 'Double jig'}
------------------- Extracted Tune 1 --------------------
{'tune': 'B|:AFDDFA|dedcBA|BGEEFA|B2ABcd|\\AFDDFA|dedcBA|BcdecA|[1d3dcB:|[2d3d2||g|f2d-ded|faaafd|cAAeAA|cBcefg|f2d-ded|faaafd|BcdecA|d3d2:|', 'K': 'D', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'E|AcABGE|cdedBG|\\AcABGE|{F}EDEGAB|AcABGE|cdedBG|{d}cBABGE|EFGA2:|(A/B/)|ceAceA|ceeedc|(B/c/d)G(B/c/d)G|BdddcB|\\cdedcB|{d}cBAGgf|ecAGED|EFGA2:|', 'K': 'A', 'M': '6/8', 'R': 'Double jig'

{'tune': 'D|D>BBD>cc|D>BBc2e|dBGFGA|BAGAFD|D>BBD>cc|D>BBc2e|dBGFGA|BGGG2:||:G|G>ggefg|fedcAF|G>gggfg|afdcAF|D>BBD>cc|D>BBc2e|dBGFGA|BGGG2:|', 'K': 'G', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'D|D>BBD>cc|D>BBc2e|dBGFGA|BAGAFD|D>BBD>cc|D>BBc2e|dBGFGA|BGGG2:||:G|G>ggefg|fedcAF|G>gggfg|afdcAF|D>BBD>cc|D>BBc2e|dBGFGA|BGGG2:|', 'K': 'G', 'M': '6/8', 'R': 'Double jig'}
------------------- Extracted Tune 101 --------------------
{'tune': 'D|D>BBD>cc|D>BBc2e|dBGFGA|BAGAFD|D>BBD>cc|D>BBc2e|dBGFGA|BGGG2:||:G|G>ggefg|fedcAF|G>gggfg|afdcAF|D>BBD>cc|D>BBc2e|dBGFGA|BGGG2:|', 'K': 'G', 'M': '6/8', 'R': 'Double jig'}
{'tune': '(c/d/)|ecAecA|ecAABc|dB=GdBG|dB=GGAB|cBcdcd|efgaza|ageedB|{d}cAAA2:||:(c/d/)|e3efd|ecAABc|d2dd=ge|dB=GGAB|cBcdcd|efga2a|ageedB|{d}cAAA2:|', 'K': 'A', 'M': '6/8', 'R': 'Double jig'}
{'tune': '(c/d/)|ecAecA|ecAABc|dB=GdBG|dB=GGAB|cBcdcd|efgaza|ageedB|{d}cAAA2:||:(c/d/)|e3efd|ecAABc|d2dd=ge|dB=GGAB|cBcdcd|efga2a|ageedB|{d}cAAA2:|', 'K': 'A', 'M': '6/8', 'R': 'Double jig'}
---------

{'tune': '(A/B/)|cBAB>cB|AGEG>AB|cBABed|BAAA2(A/B/)|cBAB>cB|AGEGAB|cdeded|cAAA2:||:(e/^f/)|g^feage|dBGG2(e/^f/)|g^feagf|e^f^ga2(e/f/)|g^feage|dBGG2(A/B/)|cBABed|cAAG2:|', 'K': 'Am', 'M': '6/8', 'R': 'Double jig'}
{'tune': '(A/B/)|cBAB>cB|AGEG>AB|cBABed|BAAA2(A/B/)|cBAB>cB|AGEGAB|cdeded|cAAA2:||:(e/^f/)|g^feage|dBGG2(e/^f/)|g^feagf|e^f^ga2(e/f/)|g^feage|dBGG2(A/B/)|cBABed|cAAG2:|', 'K': 'Am', 'M': '6/8', 'R': 'Double jig'}
------------------- Extracted Tune 153 --------------------
{'tune': '(A/B/)|cBAB>cB|AGEG>AB|cBABed|BAAA2(A/B/)|cBAB>cB|AGEGAB|cdeded|cAAA2:||:(e/^f/)|g^feage|dBGG2(e/^f/)|g^feagf|e^f^ga2(e/f/)|g^feage|dBGG2(A/B/)|cBABed|cAAG2:|', 'K': 'Am', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'E|A2EAce|edeABA|=G2DG>Bc|dcd=G2B|A2EABd|e2f=gfg|edcBcd|ecAA2:||:a|agaA2a|agaA2=g|=gfg=G2g|=gfg=G2B|c2cd2d|e2f=gfg|edcBcd|ecAA2:|', 'K': 'A', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'E|A2EAce|edeABA|=G2DG>Bc|dcd=G2B|A2EABd|e2f=gfg|edcBcd|ecAA2:||:a|agaA2a|agaA2=g|=gfg=G2g|=gfg=G2B|c2cd2d|e2f

{'tune': 'DGGBGG|dGGBGG|DGGBAG|FAABGE|DGGBGG|dGGBGG|DGGABc|[1BGAGED:|[2BGGG2||(3A/B/c/|d2gB2d|(A/B/c).ABGE|DGG{c}BAG|FAAABc|d2gB2d|(A/B/c).ABGE|DGGABc|BGGGA(B/c/)|dgfedc|(A/B/c).ABGE|DGG{c}BAG|FAAABc|dgfedc|(A/B/c).ABGE|DGGABc|BGGG2|]', 'K': 'G', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'DGGBGG|dGGBGG|DGGBAG|FAABGE|DGGBGG|dGGBGG|DGGABc|[1BGAGED:|[2BGGG2||(3A/B/c/|d2gB2d|(A/B/c).ABGE|DGG{c}BAG|FAAABc|d2gB2d|(A/B/c).ABGE|DGGABc|BGGGA(B/c/)|dgfedc|(A/B/c).ABGE|DGG{c}BAG|FAAABc|dgfedc|(A/B/c).ABGE|DGGABc|BGGG2|]', 'K': 'G', 'M': '6/8', 'R': 'Double jig'}
------------------- Extracted Tune 193 --------------------
{'tune': 'DGGBGG|dGGBGG|DGGBAG|FAABGE|DGGBGG|dGGBGG|DGGABc|[1BGAGED:|[2BGGG2||(3A/B/c/|d2gB2d|(A/B/c).ABGE|DGG{c}BAG|FAAABc|d2gB2d|(A/B/c).ABGE|DGGABc|BGGGA(B/c/)|dgfedc|(A/B/c).ABGE|DGG{c}BAG|FAAABc|dgfedc|(A/B/c).ABGE|DGGABc|BGGG2|]', 'K': 'G', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'D|GFGdBG|BABdBG|GFGdBG|GAGGED|GFGdBG|BABdBG|GBGB2A|GEEE2:||:d|gfgaga|bgeedB|gfgaga|bgee2(g/a/)|b

{'tune': 'B|cAdcAG|EGEGAB|cAdcAG|EAAA2B|cAdcAG|EGEGAB|cde{g}fed|cAAA2:||:(e/^f/)|gagged|cABcde|eaaage|eddd2(e/^f/)|gagged|cABcde|fedcAG|EAAA2:|', 'K': 'Am', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'B|cAdcAG|EGEGAB|cAdcAG|EAAA2B|cAdcAG|EGEGAB|cde{g}fed|cAAA2:||:(e/^f/)|gagged|cABcde|eaaage|eddd2(e/^f/)|gagged|cABcde|fedcAG|EAAA2:|', 'K': 'Am', 'M': '6/8', 'R': 'Double jig'}
------------------- Extracted Tune 221 --------------------
{'tune': 'B|cAdcAG|EGEGAB|cAdcAG|EAAA2B|cAdcAG|EGEGAB|cde{g}fed|cAAA2:||:(e/^f/)|gagged|cABcde|eaaage|eddd2(e/^f/)|gagged|cABcde|fedcAG|EAAA2:|', 'K': 'Am', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'A,|D>EDF>ED|A>BAc>AG|ABAA2G|FAFGEC|D>EDF>ED|A>BAc>AG|AGFEDE|FDDD2:||:A|d2ef>ed|f2gagf|cefgec|gecgec|dedfed|cedcAG|AGFEDE|FDDD2:|', 'K': 'Dm', 'M': '6/8', 'R': 'Double jig'}
{'tune': 'A,|D>EDF>ED|A>BAc>AG|ABAA2G|FAFGEC|D>EDF>ED|A>BAc>AG|AGFEDE|FDDD2:||:A|d2ef>ed|f2gagf|cefgec|gecgec|dedfed|cedcAG|AGFEDE|FDDD2:|', 'K': 'Dm', 'M': '6/8', 'R': 'Double jig'}
----------

...

------------ Serializing example 329 -------------
{'tune': '(A/G/)|(TFEF)DA,A,|DFAdAF|GdGFGA|BGEEFG|(TFEF)DA,A,|(.DF(3A/B/c/d)AF|GdGEFG|FDDD2:||:(3b/a/g/|fdfece|dAGFED|(DF(3A/B/c/d)AF|GBGEFG|fdaece|dAGFED|DFdEFG|FDDD2::A|(3.d/.d/.d/ddfdd|addfdd|(B/c/d/e/f)Tg2e|fdaecA|[1Addfdd|addefg|EFGdAG|FDDD2:|[2(a/g/f).ageg|fdaecA|EFGdAG|FDDD2|]', 'K': 'D', 'M': '6/8', 'R': 'Double jig'}
['<s>', 'M:6/8', 'K:D', '(', 'A', 'G', ')', '|', '(', 'F', 'E', 'F', ')', 'D', 'A,', 'A,', '|', 'D', 'F', 'A', 'd', 'A', 'F', '|', 'G', 'd', 'G', 'F', 'G', 'A', '|', 'B', 'G', 'E', 'E', 'F', 'G', '|', '(', 'F', 'E', 'F', ')', 'D', 'A,', 'A,', '|', '(', '.', 'D', 'F', '(3', 'A', 'B', 'c', 'd', ')', 'A', 'F', '|', 'G', 'd', 'G', 'E', 'F', 'G', '|', 'F', 'D', 'D', 'D', '2', ':|', '|:', '(3', 'b', 'a', 'g', '|', 'f', 'd', 'f', 'e', 'c', 'e', '|', 'd', 'A', 'G', 'F', 'E', 'D', '|', '(', 'D', 'F', '(3', 'A', 'B', 'c', 'd', ')', 'A', 'F', '|', 'G', 'B', 'G', 'E', 'F', 'G', '|', 'f', 'd', 'a', 'e', 'c', 'e', '|', 'd', 'A'

------------ Serializing example 337 -------------
{'tune': 'G|FEFAFD|dcdecA|fedgec|dcdecA|FEFAFA|dfaece|fedge(c|A)ddd2:|||c|ecAefg|afdbge|afdgec|(d/e/f)(de)cA|FEFAFA|(d/e/f)(ae)cA|fafge(c|e)ddd2:|||g|afddgf|ecAAfg|afdBg(f|e)fd(cB)A|afd(dcd)|(gfg)ece|fedge(c|e)ddd2:|||B|AFDDFA|(dcd)AFD|AFDDEF|(G/A/B)(GE)FG|ABADFA|(d/e/f)(ae)ce|fafge(c|e)ddd2:|', 'K': 'D', 'M': '6/8', 'R': 'Double jig'}
['<s>', 'M:6/8', 'K:D', 'G', '|', 'F', 'E', 'F', 'A', 'F', 'D', '|', 'd', 'c', 'd', 'e', 'c', 'A', '|', 'f', 'e', 'd', 'g', 'e', 'c', '|', 'd', 'c', 'd', 'e', 'c', 'A', '|', 'F', 'E', 'F', 'A', 'F', 'A', '|', 'd', 'f', 'a', 'e', 'c', 'e', '|', 'f', 'e', 'd', 'g', 'e', '(', 'c', '|', 'A', ')', 'd', 'd', 'd', '2', ':|', '||', 'c', '|', 'e', 'c', 'A', 'e', 'f', 'g', '|', 'a', 'f', 'd', 'b', 'g', 'e', '|', 'a', 'f', 'd', 'g', 'e', 'c', '|', '(', 'd', 'e', 'f', ')', '(', 'd', 'e', ')', 'c', 'A', '|', 'F', 'E', 'F', 'A', 'F', 'A', '|', '(', 'd', 'e', 'f', ')', '(', 'a', 'e', ')', 'c', 'A', '|', 'f', 'a', 'f', 

------------ Serializing example 345 -------------
{'tune': 'g|f2dedc|d2BAGF|GABdA(F|G)EEE2g|f2dedc|d2BAGF|GABdA(G|F)DDD2:|g|afdded|gfg(a2f)|g2abaf|gee(ef)g|[1afdde(d|g)fga2f|g2abag|fddd2:|[2(f2d)(e2c)|d2(BA)GF|GABdA(G|F)DDD2|]', 'K': 'D', 'M': '6/8', 'R': 'Double jig'}
['<s>', 'M:6/8', 'K:D', 'g', '|', 'f', '2', 'd', 'e', 'd', 'c', '|', 'd', '2', 'B', 'A', 'G', 'F', '|', 'G', 'A', 'B', 'd', 'A', '(', 'F', '|', 'G', ')', 'E', 'E', 'E', '2', 'g', '|', 'f', '2', 'd', 'e', 'd', 'c', '|', 'd', '2', 'B', 'A', 'G', 'F', '|', 'G', 'A', 'B', 'd', 'A', '(', 'G', '|', 'F', ')', 'D', 'D', 'D', '2', ':|', 'g', '|', 'a', 'f', 'd', 'd', 'e', 'd', '|', 'g', 'f', 'g', '(', 'a', '2', 'f', ')', '|', 'g', '2', 'a', 'b', 'a', 'f', '|', 'g', 'e', 'e', '(', 'e', 'f', ')', 'g', '|', '[', 'a', 'f', 'd', 'd', 'e', '(', 'd', '|', 'g', ')', 'f', 'g', 'a', '2', 'f', '|', 'g', '2', 'a', 'b', 'a', 'g', '|', 'f', 'd', 'd', 'd', '2', ':|', '[', '2', '(', 'f', '2', 'd', ')', '(', 'e', '2', 'c', ')', '|', 'd', '2', '('

------------ Serializing example 353 -------------
{'tune': 'B|AFDDFD|DFD(TB2A)|BcdAFA|BEEE2(d/B/)|AFDDFD|DFD(TB2A)|BcdAFA|FDDD2:|g|fefd2e|fga(TB2A)|BcdAFA|BEEE2g|fefd2e|fga(TB2A)|BcdAFA|FDDD2g|fefd2e|fga(TB2A)|BcdAFA|BEEE2g|fgaefg|fea(TB2A)|Bcd(TF2E)|FDDD2||', 'K': 'D', 'M': '6/8', 'R': 'Double jig'}
['<s>', 'M:6/8', 'K:D', 'B', '|', 'A', 'F', 'D', 'D', 'F', 'D', '|', 'D', 'F', 'D', '(', 'B', '2', 'A', ')', '|', 'B', 'c', 'd', 'A', 'F', 'A', '|', 'B', 'E', 'E', 'E', '2', '(', 'd', 'B', ')', '|', 'A', 'F', 'D', 'D', 'F', 'D', '|', 'D', 'F', 'D', '(', 'B', '2', 'A', ')', '|', 'B', 'c', 'd', 'A', 'F', 'A', '|', 'F', 'D', 'D', 'D', '2', ':|', 'g', '|', 'f', 'e', 'f', 'd', '2', 'e', '|', 'f', 'g', 'a', '(', 'B', '2', 'A', ')', '|', 'B', 'c', 'd', 'A', 'F', 'A', '|', 'B', 'E', 'E', 'E', '2', 'g', '|', 'f', 'e', 'f', 'd', '2', 'e', '|', 'f', 'g', 'a', '(', 'B', '2', 'A', ')', '|', 'B', 'c', 'd', 'A', 'F', 'A', '|', 'F', 'D', 'D', 'D', '2', 'g', '|', 'f', 'e', 'f', 'd', '2', 'e', '|', 'f', 'g

------------ Serializing example 363 -------------
{'tune': 'B|cBABA(G|A)GE(GE)D|cBAdB(G|E)AAA2B|cBABA(G|A)GE(GE)(D|E)FETE2(D|E)AAA2:||:B|cBcdc(d|e)cA(GE)D|cBcd=f(d|c)ABTc2d|edcdcB|(cB)(AG)E(D|E)FETE2(D|E)AAA2:|', 'K': 'Ador', 'M': '6/8', 'R': 'Double jig'}
['<s>', 'M:6/8', 'K:Ador', 'B', '|', 'c', 'B', 'A', 'B', 'A', '(', 'G', '|', 'A', ')', 'G', 'E', '(', 'G', 'E', ')', 'D', '|', 'c', 'B', 'A', 'd', 'B', '(', 'G', '|', 'E', ')', 'A', 'A', 'A', '2', 'B', '|', 'c', 'B', 'A', 'B', 'A', '(', 'G', '|', 'A', ')', 'G', 'E', '(', 'G', 'E', ')', '(', 'D', '|', 'E', ')', 'F', 'E', 'E', '2', '(', 'D', '|', 'E', ')', 'A', 'A', 'A', '2', ':|', '|:', 'B', '|', 'c', 'B', 'c', 'd', 'c', '(', 'd', '|', 'e', ')', 'c', 'A', '(', 'G', 'E', ')', 'D', '|', 'c', 'B', 'c', 'd', '=f', '(', 'd', '|', 'c', ')', 'A', 'B', 'c', '2', 'd', '|', 'e', 'd', 'c', 'd', 'c', 'B', '|', '(', 'c', 'B', ')', '(', 'A', 'G', ')', 'E', '(', 'D', '|', 'E', ')', 'F', 'E', 'E', '2', '(', 'D', '|', 'E', ')', 'A', 'A', 'A', '2', ':

In [4]:
dataset = preprocessor.load_tfrecord_dataset()
print(dataset)

<MapDataset shapes: ({K: (), M: (), R: (), tune_length: ()}, {input: (None, None), output: (None, None)}), types: ({K: tf.int64, M: tf.int64, R: tf.int64, tune_length: tf.int64}, {input: tf.int64, output: tf.int64})>
<MapDataset shapes: ({K: (), M: (), R: (), tune_length: ()}, {input: (None, None), output: (None, None)}), types: ({K: tf.int64, M: tf.int64, R: tf.int64, tune_length: tf.int64}, {input: tf.int64, output: tf.int64})>
